### Market Basket Analysis on grocery store data

Dataset:  Groceries_dataset

        Member_number   the number of cutomer
        Date            purchase date
        itemDescriptio  product name


In [1]:
# import library and read the dataset
import pandas as pd 
grocery_df = pd.read_csv("c:/Users/Windows/Downloads/data/groceries_dataset.csv")

In [2]:
# check the data structure
grocery_df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [4]:
# check the size of this dataset
grocery_df.shape

(38765, 3)

In [5]:
# check the duplicates
grocery_df.duplicated().sum()

759

In [6]:
# remove duplicates
grocery_df1 = grocery_df.drop_duplicates().copy()

In [7]:
# make sure the duplcatea has been removed successfully 
grocery_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38006 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38006 non-null  int64 
 1   Date             38006 non-null  object
 2   itemDescription  38006 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [9]:
grocery_df1.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [10]:
# cereate unique transaction ID by concatenating Member_number and Date
grocery_df1["transaction_id"] = grocery_df1["Member_number"].astype("str") + grocery_df1["Date"]

In [11]:
# check transaction_id has been created successfully
grocery_df1.head()

,Member_number,Date,itemDescription,transaction_id
0,1808,21-07-2015,tropical fruit,180821-07-2015
1,2552,05-01-2015,whole milk,255205-01-2015
2,2300,19-09-2015,pip fruit,230019-09-2015
3,1187,12-12-2015,other vegetables,118712-12-2015
4,3037,01-02-2015,whole milk,303701-02-2015


In [13]:
# How many transactions in total
grocery_df1["transaction_id"].value_counts().shape

(14963,)

In [14]:
# agggregrate all differnt purchosed product under each transaction ID
grocery_groups = grocery_df1.groupby("transaction_id")["itemDescription"]
basket_new = grocery_groups.unique()

In [16]:
# needs the product combination only, ignore transaction ID
my_basket = basket_new.values

In [22]:
my_basket.shape

(14963,)

In [27]:
# using TransactionEncoder to create datefrome for each product occurance 
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
basket = te.fit_transform(my_basket)
basket_df = pd.DataFrame(basket, columns=te.columns_)

In [58]:
# check the data structure
basket_df

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14958,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
14959,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14960,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14961,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [53]:
# using Apriori to focus on the itemset that occured over 0.1%
#  ess frequent items will excluded
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
x = apriori(basket_df,min_support=0.001, use_colnames=True, max_len =167)

In [59]:
# check the data structure
x

,support,itemsets
0,0.004010,(Instant food products)
1,0.021386,(UHT-milk)
2,0.001470,(abrasive cleaner)
3,0.001938,(artif. sweetener)
4,0.008087,(baking powder)
...,...,...
745,0.001136,"(whole milk, sausage, rolls/buns)"
746,0.001002,"(soda, whole milk, rolls/buns)"
747,0.001337,"(whole milk, rolls/buns, yogurt)"
748,0.001069,"(soda, whole milk, sausage)"


In [57]:
#  create associate ruls for the metric "Lift"
association_rules(x,metric="lift",min_threshold=1).sort_values("lift", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
238,(sausage),"(whole milk, yogurt)",0.060349,0.011161,0.001470,0.024363,2.182917,7.967480e-04,1.013532
235,"(whole milk, yogurt)",(sausage),0.011161,0.060349,0.001470,0.131737,2.182917,7.967480e-04,1.082219
234,"(whole milk, sausage)",(yogurt),0.008955,0.085879,0.001470,0.164179,1.911760,7.012151e-04,1.093681
239,(yogurt),"(whole milk, sausage)",0.085879,0.008955,0.001470,0.017121,1.911760,7.012151e-04,1.008307
87,(specialty chocolate),(citrus fruit),0.015973,0.053131,0.001403,0.087866,1.653762,5.548137e-04,1.038081
...,...,...,...,...,...,...,...,...,...
145,(grapes),(soda),0.014436,0.097106,0.001403,0.097222,1.001195,1.674919e-06,1.000129
5,(beef),(citrus fruit),0.033950,0.053131,0.001804,0.053150,1.000349,6.297697e-07,1.000020
4,(citrus fruit),(beef),0.053131,0.033950,0.001804,0.033962,1.000349,6.297697e-07,1.000012
138,(rolls/buns),(fruit/vegetable juice),0.110005,0.034017,0.003743,0.034022,1.000136,5.091755e-07,1.000005


In [61]:
# Beased on the calculation, th Lift of antecedent and consequent is resersible.
# By showing top 20 datapoints we can see Top 10 product or product combination that have the hightest 10 Lift score.

association_rules(x,metric="lift",min_threshold=1).sort_values("lift", ascending=False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
238,(sausage),"(whole milk, yogurt)",0.060349,0.011161,0.001470,0.024363,2.182917,0.000797,1.013532
235,"(whole milk, yogurt)",(sausage),0.011161,0.060349,0.001470,0.131737,2.182917,0.000797,1.082219
234,"(whole milk, sausage)",(yogurt),0.008955,0.085879,0.001470,0.164179,1.911760,0.000701,1.093681
239,(yogurt),"(whole milk, sausage)",0.085879,0.008955,0.001470,0.017121,1.911760,0.000701,1.008307
87,(specialty chocolate),(citrus fruit),0.015973,0.053131,0.001403,0.087866,1.653762,0.000555,1.038081
86,(citrus fruit),(specialty chocolate),0.053131,0.015973,0.001403,0.026415,1.653762,0.000555,1.010726
236,"(sausage, yogurt)",(whole milk),0.005748,0.157923,0.001470,0.255814,1.619866,0.000563,1.131541
237,(whole milk),"(sausage, yogurt)",0.157923,0.005748,0.001470,0.009310,1.619866,0.000563,1.003596
123,(tropical fruit),(flour),0.067767,0.009757,0.001069,0.015779,1.617141,0.000408,1.006118
122,(flour),(tropical fruit),0.009757,0.067767,0.001069,0.109589,1.617141,0.000408,1.046969


Conclusion:
By sorting the metric Lift descendingly, we can see what produdct or products combination occur together than expected if 
their purchase frequencies were independent.
In this case, (sausage) and 	(whole milk, yogurt) have highest Lift score. So we can promote them together in marketing 
campaigns to drive up more sales.

Other favorable product or product combinations are
(whole milk, sausage)&(yogurt)	
(specialty chocolate)& (citrus fruit)
(sausage, yogurt)	& (whole milk)	
(tropical fruit) & (flour)
(beverages)	& (sausage)	
(sausage) & (soda, whole milk)	
(napkins)	& (pastry)	
(processed cheese)	& (root vegetables)	
(pip fruit)	(hard cheese)	